In [28]:
import gurobipy as gp
import numpy as np
rng = np.random.default_rng(42)

# Knapsack Problem (2.3.1)
$$
\begin{align*}
    \max\ &z = \sum_j c_j y_j \\
    \text{s.t.} &\sum_j a_j y_j \le b \\
    &y_j = 0 \text{ or } 1 & j = 1, 2, \dots, n
\end{align*}
$$

In [29]:
knapsack = gp.Model()
n_var = 10

x = [knapsack.addVar(vtype=gp.GRB.BINARY) for j in range(n_var)]

c = [rng.uniform(0, 10) for j in range(n_var)]
objective = gp.quicksum(c[j]*x[j] for j in range(n_var))

a = [rng.uniform(0, 10) for j in range(n_var)]
b = rng.uniform(n_var, 3*n_var)
constraint = gp.quicksum(a[j]*x[j] for j in range(n_var)) <= b

knapsack.setObjective(objective, gp.GRB.MAXIMIZE)
knapsack.addConstr(constraint)

knapsack.optimize()

print(f"Solution vector: {[var.X for var in x]}")

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 10 columns and 10 nonzeros
Model fingerprint: 0x1db5e585
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [6e-01, 9e+00]
  Objective range  [9e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 3e+01]
Found heuristic solution: objective 29.5167406
Presolve time: 0.00s
Presolved: 1 rows, 10 columns, 10 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 4.708557e+01, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   47.08557    0    1   29.51674   47.08557  59.5%     -    0s
H    0     0                      46.0576627   47.08557  2.23%     -    0s
     0     0   47.08557    0    

# Capacitated Lot Sizing (2.4.2)
## Decision Variables
$y_t$: whether or not to produce in a time period.  
$x_t$: how much to produce in a time period.  
## Slack Variables
$s_t$: inventory level at the end of each time period ($s_0 = 0$).  
## Input Variables
$d_t$: demand in each time period.  
$f_t$: fixed setup cost in a time period.  
$c_t$: unit production cost.  
$h_t$: unit holding cost.  
$u_t$: production capacity for a time period.
$$
\begin{align*}
    \min\ &\sum_t (c_t x_t + f_t y_t + h_t s_t) \\
    \text{s.t.} &s_{t-1} + x_t + s_t = d_t \\
    &x_t \le u_t y_t \\
    &x_t \ge 0 \\
    &s_t \ge 0 \\
    &y_t = 0 \text{ or } 1 \\
\end{align*}
$$

In [32]:
cap_lot_size = gp.Model()
n_time_step = 10

y = [cap_lot_size.addVar(vtype=gp.GRB.BINARY) for t in range(n_time_step)]
x = [cap_lot_size.addVar(vtype=gp.GRB.CONTINUOUS) for t in range(n_time_step)]
s = [cap_lot_size.addVar(vtype=gp.GRB.CONTINUOUS) for t in range(n_time_step)]

c = [rng.uniform(0, 10) for t in range(n_time_step)]
f = [rng.uniform(0, 10) for t in range(n_time_step)]
h = [rng.uniform(0, 10) for t in range(n_time_step)]
d = [rng.uniform(0, 10) for t in range(n_time_step)]
u = [rng.uniform(5, 10) for t in range(n_time_step)]

objective = gp.quicksum(c[t]*x[t]+f[t]*y[t]+h[t]*s[t] for t in range(n_time_step))
cap_lot_size.setObjective(objective, gp.GRB.MINIMIZE)

demand_lhs = [x[0] - s[0]] + [s[t-1] + x[t] - s[t] for t in range(1, n_time_step)]
demand_constraints = [demand_lhs[t] == d[t] for t in range(n_time_step)]
capacity_constraints = [x[t] <= u[t]*y[t] for t in range(n_time_step)]
constraints = demand_constraints + capacity_constraints
for constraint in constraints:
    cap_lot_size.addConstr(constraint)

cap_lot_size.optimize()

print(f"Lot Sizes: {[var.X for var in x]}")
print(f"Surplus Sizes: {[var.X for var in s]}")

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 20 rows, 30 columns and 49 nonzeros
Model fingerprint: 0xc0533705
Variable types: 20 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolved: 13 rows, 20 columns, 32 nonzeros
Variable types: 13 continuous, 7 integer (7 binary)

Root relaxation: objective 2.352259e+02, 13 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  235.22591    0    4          -  235.22591      -     -    0s
H    0     0                     247.8935701  235.22591  5.11%     -    0s

Cutting planes:
  Gomory: 4
  Implie